In [ ]:
from pepars.analysis import DNA as DNA_analysis
from pepars.utils import DNA as DNA_utils

from pepars.plotting import plotting
plotting.init_notebook_mode()
from pepars.plotting import DNA as DNA_plotting

from pepars.analysis import amino_acids as AA_analysis
from pepars.analysis import statistics as virus_stats
import pandas as pd

from pepars.analysis import statistics

In [ ]:
#R2 DNA vs oligopool
workspace_directory = "example_data/" 
# Sample to base amino acid bias off of. If None, will use template probabilities instead
input_file = workspace_directory + "random_input_sequence_counts.xlsx"    #set your file name
input_df = pd.read_excel(input_file) 
input_df.head()

In [ ]:
output_file = workspace_directory + "random_output_sequence_counts.xlsx"    #set your file name
output_df = pd.read_excel(output_file) 
output_df.head()

In [ ]:
sequence_counts_recovered = {}

#call columns from output lib
for row in output_df.iterrows():
    sequence = row[1]["Sequence"]
    count = row[1]["Read Count"]
    
    if sequence in sequence_counts_recovered:
        sequence_counts_recovered[sequence] += count
    else:
        sequence_counts_recovered[sequence] = count
        
sample_amino_acid_counts_by_position_outputlib = AA_analysis.get_amino_acid_counts_by_position(sequence_counts_recovered)

In [ ]:
sequence_counts_starting = {}
#call columns from R2 synth virus file
for row in input_df.iterrows():
    sequence = row[1]["Sequence"]
    count = row[1]["Read Count"]
    
    if sequence in sequence_counts_starting:
        sequence_counts_starting[sequence] += count
    else:
        sequence_counts_starting[sequence] = count
        
sample_amino_acid_counts_by_position_inputlib = AA_analysis.get_amino_acid_counts_by_position(sequence_counts_starting)

In [ ]:
# Set the p value threshold for plotting
P_VALUE_THRESHOLD = 1e-4

# Whether to invert the outline by crosshatch (True) or not (False)
INVERT_OUTLINE = False

In [ ]:
#plot signifance heatmap
p_values, z_scores = virus_stats.get_significance_of_amino_acid_ratios(
    sample_amino_acid_counts_by_position_outputlib, #set the numerator of the ratio
    sample_amino_acid_counts_by_position_inputlib,  #set the denominator of the ratio
    test_type=virus_stats.Test_Type.BINOMIAL_NORMAL_APPROXIMATION_LOG2,   #set the function
    multiple_comparison_correction=True
)

figure=DNA_plotting.plot_signficant_amino_acid_biases(
    z_scores.values,
    p_values=p_values.values,
    p_value_threshold=P_VALUE_THRESHOLD,
    interactive=True,
    invert_outline=INVERT_OUTLINE
)

In [ ]:
#use this if a specific input template is required
starting_amino_acid_NNK = DNA_analysis.get_amino_acid_probabilities_from_template('NNKNNKNNKNNKNNKNNKNNK')

#plot signifance heatmap
p_values, z_scores = virus_stats.get_significance_of_amino_acid_ratios(
    sample_amino_acid_counts_by_position_outputlib, #set the numerator of the ratio
    starting_amino_acid_NNK,  #set the denominator of the ratio
    test_type=virus_stats.Test_Type.BINOMIAL_NORMAL_APPROXIMATION_LOG2,   #set the function
    multiple_comparison_correction=True
)

figure=DNA_plotting.plot_signficant_amino_acid_biases(
    z_scores.values,
    p_values=p_values.values,
    p_value_threshold=P_VALUE_THRESHOLD,
    interactive=True,
    invert_outline=INVERT_OUTLINE
)